In [ ]:
!which python
!python -V

/Users/pandysudhan/dev/projects/doe-data/venv/bin/python
Python 3.12.1


In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO


## Sensor location dataframe

# Data tables


In [ ]:
import os

class DataObject:
    def __init__(self, folder_path):
        self.folder_path = folder_path

        self.vulink_df = None
        self.at500_df = None
        self.at200_df = None
        self.parse_files_to_df()

    def parse_files_to_df(self):
        vulink_dfs = []
        at500_dfs = []
        at200_dfs = []

        count = 0
        
        #  sensor location data easting and northing
        easting_northing = pd.read_csv("../data/Rel. Coords.xlsx - Sheet1.csv")
        
        # Iterate over each file in the folder
        for filename in os.listdir(self.folder_path):
            
            if filename.endswith('.htm'):
                
                location_prop = {}
                '''
                Location Property contains:
                    'Location Name ': ' FSB 130D (AT500)',
                    'Location ID ': ' 5191429905973248',
                    'Latitude ': ' 40.58833 °',
                    'Longitude ': ' -105.06587 °',
                    'Time shown ': ' UTC'}
                '''

                
                print("\n" + str(count) + " Reading File: " + filename)
                file_path = os.path.join(self.folder_path, filename)
                with open(file_path, 'r') as f:
                    html_content = f.read()
                soup = BeautifulSoup(html_content, 'html.parser')
                table = soup.find('table')
                df = pd.read_html(StringIO(str(table)))[0]

                # meta data part
                for i in range(1, 6):
                    s = df[0][i]
                    if "=" in s:
                        temp_ = s.split('=')
                    else:
                        temp_ = s.split('in')
                    location_prop[temp_[0]] = temp_[1]

                if len(df) >= 9:
                    df = pd.DataFrame(df.values[9:], columns=df.iloc[8])
                    df.reset_index(drop=True, inplace=True)
                else:
                    print("\t Error: File doesn't have enough rows to proceed with data extraction.")
                    df = None  # Or handle this case as needed


                 # Add Location ID to each row
                splitted_name = filename.split('_')
                station_id =  splitted_name[1] + "_" + splitted_name[2]
                               
                if station_id is not None:
                    if df is not None:
                        df['station_id'] = station_id


                
                
                # Identify the type of file and append to the respective list
                if 'vulink' in filename.lower():
                    vulink_dfs.append(df)
                elif 'at500' in filename.lower():
                    at500_dfs.append(df)
                elif 'at200' in filename.lower():
                    at200_dfs.append(df)

            count += 1


        print("\n")
        
        if vulink_dfs:
            self.vulink_dfs = pd.concat(vulink_dfs, ignore_index=True)

            # remove duplicates
            rows_count = len(self.vulink_dfs)
            self.vulink_dfs.drop_duplicates(inplace=True)
            duplicates = rows_count - len(self.vulink_dfs)
            print("Dropped " + str(duplicates) + " duplicate rows in Vulink")

        
        if at500_dfs:
            self.at500_dfs = pd.concat(at500_dfs, ignore_index=True)

            
            # remove duplicates
            rows_count = len(self.at500_dfs)
            self.at500_dfs.drop_duplicates(inplace=True)
            duplicates = rows_count - len(self.at500_dfs)
            print("Dropped " + str(duplicates) + " duplicate rows in at500")

        
        if at200_dfs:
            self.at200_dfs = pd.concat(at200_dfs, ignore_index=True)

            # remove duplicates
            rows_count = len(self.at200_dfs)
            self.at200_dfs.drop_duplicates(inplace=True)
            duplicates = rows_count - len(self.at200_dfs)
            print("Dropped " + str(duplicates) + " duplicate rows in at200")


        # merge the easting and northing data with the corresponding sensro data
        self.vulink_dfs = pd.merge(self.vulink_dfs, easting_northing, on='station_id', how='left')
        self.at500_dfs = pd.merge(self.at500_dfs, easting_northing, on='station_id', how='left')
        self.at200_dfs = pd.merge(self.at200_dfs, easting_northing, on='station_id', how='left')
        


In [ ]:
obj = DataObject(folder_path= "../data/")


0 Reading File: HydroVu_FSB_128D_(VuLink)_2022-10-31_05-00-00_Export.htm

1 Reading File: HydroVu_FSB_118D_(VuLink)_2023-04-01_05-00-00_Export.htm

2 Reading File: HydroVu_FSB_138D_(AT200)_2023-04-01_05-00-00_Export.htm

3 Reading File: HydroVu_FSB_099D_(VuLink)_2022-10-31_05-00-00_Export.htm

4 Reading File: HydroVu_FPZ_06A_(VuLink)_2023-04-01_05-00-00_Export.htm

5 Reading File: HydroVu_FSB_078_(VuLink)_2022-10-31_05-00-00_Export.htm

6 Reading File: HydroVu_FSB_135D_(VuLink)_2023-04-01_05-00-00_Export.htm

7 Reading File: HydroVu_FSB_079_(VuLink)_2022-10-31_05-00-00_Export.htm

8 Reading File: HydroVu_FPZ_06B_(AT200)_2023-04-01_05-00-00_Export.htm

9 Reading File: HydroVu_FPZ_04A_(AT500)_2022-10-31_05-00-00_Export.htm

10 Reading File: HydroVu_FSB_126_(AT500)_2022-10-31_05-00-00_Export.htm

11 Reading File: HydroVu_FSB_124D_(AT200)_2022-10-31_05-00-00_Export.htm

12 Reading File: HydroVu_FSB_138D_(VuLink)_2022-10-31_05-00-00_Export.htm

13 Reading File: HydroVu_FPZ_08AR_(AT200)_202

In [ ]:
display(obj.vulink_dfs)
display(obj.at500_dfs)
display(obj.at200_dfs)



,Date Time,Battery Level (%),Baro (psi),station_id,Temperature (C),Pulse (Hz),easting,northing
0,2022-11-03 18:15:00.000,97.7552719116211,NaN,FSB_128D,NaN,NaN,1382733.504,3875766.69
1,2022-11-03 19:00:00.000,97.72055053710938,NaN,FSB_128D,NaN,NaN,1382733.504,3875766.69
2,2022-11-03 20:00:00.000,97.71411895751953,NaN,FSB_128D,NaN,NaN,1382733.504,3875766.69
3,2022-11-03 21:00:00.000,97.70768737792969,NaN,FSB_128D,NaN,NaN,1382733.504,3875766.69
4,2022-11-03 22:00:00.000,97.701904296875,NaN,FSB_128D,NaN,NaN,1382733.504,3875766.69
...,...,...,...,...,...,...,...,...
345571,2023-10-31 00:00:00.000,54.721580505371094,14.594597149396641,FSB_097D,NaN,NaN,1383076.160,3876266.79
345572,2023-10-31 01:00:00.000,54.69328689575195,14.5949760010627,FSB_097D,NaN,NaN,1383076.160,3876266.79
345573,2023-10-31 02:00:00.000,54.681068420410156,14.595138871872406,FSB_097D,NaN,NaN,1383076.160,3876266.79
345574,2023-10-31 03:00:00.000,54.66756820678711,14.593776894231812,FSB_097D,NaN,NaN,1383076.160,3876266.79


,Date Time,Actual Conductivity (µS/cm),Specific Conductivity (µS/cm),Salinity (psu),Resistivity (Ω-cm),Density (g/cm³),Total Dissolved Solids (mg/L),pH (pH),pH MV (mV),ORP (mV),...,Partial Pressure O₂ (psi),Temperature (C),External Voltage (V),Pressure (psi),Depth (m),Level: Depth to Water (m),station_id,Level: Elevation (m),easting,northing
0,2023-01-11 14:34:00.000,161.13510131835938,169.81427001953125,0.08978159725666046,6205.97265625,0.9987892508506775,110.37927865982056,6.345862865447998,33.32376480102539,242.2526397705078,...,1.7288530268441515,17.324094772338867,15.78600025177002,1.5765867233276367,1.2999485595703126,2.2649719230651857,FPZ_04A,NaN,1383303.353,3875869.513
1,2023-01-11 14:39:00.000,161.5494842529297,169.9202117919922,0.08982338756322861,6190.0537109375,0.9987720251083374,110.44813692569733,6.392928123474121,30.76314926147461,241.0128631591797,...,1.653776381115175,17.420801162719727,15.812999725341797,1.5758419036865234,1.2994010639190674,2.1770143558502197,FPZ_04A,NaN,1383303.353,3875869.513
2,2023-01-12 00:00:00.000,160.56321716308594,167.5901336669922,0.088505320250988,6228.07666015625,0.9987016320228577,108.93359035253525,6.545793533325195,22.439956665039062,233.12835693359375,...,1.1215573641598027,17.80475425720215,15.777000427246094,1.5779380798339844,1.3009146343231202,2.1755404632568363,FPZ_04A,NaN,1383303.353,3875869.513
3,2023-01-13 00:00:00.000,160.4653778076172,167.549560546875,0.08848636597394943,6231.8740234375,0.9987050294876099,108.90721529722214,6.533608913421631,23.105144500732422,233.1378936767578,...,1.0534621067788046,17.78632926940918,15.803999900817871,1.5864925384521484,1.3068721195220947,2.1695261501312255,FPZ_04A,NaN,1383303.353,3875869.513
4,2023-01-14 00:00:00.000,160.04112243652344,167.14593505859375,0.08827055990695953,6248.39404296875,0.9987069964408875,108.64485800266266,6.524962902069092,23.577213287353516,230.4080047607422,...,1.0645632850852904,17.774520874023438,15.78600025177002,1.5920772552490234,1.3107986240386964,2.16559979095459,FPZ_04A,NaN,1383303.353,3875869.513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104847,2024-03-03 01:00:00.000,NaN,NaN,0.09241610020399094,NaN,NaN,114.03341591358185,4.379131317138672,142.17138671874997,NaN,...,2.2958317527099736,NaN,NaN,NaN,NaN,NaN,FSB_130D,NaN,1383264.637,3875923.874
104848,2024-03-03 02:00:00.000,NaN,NaN,0.09241019189357758,NaN,NaN,114.02643471956253,4.3795270919799805,142.14993286132812,NaN,...,2.2971344239364018,NaN,NaN,NaN,NaN,NaN,FSB_130D,NaN,1383264.637,3875923.874
104849,2024-03-03 03:00:00.000,NaN,NaN,0.09242653846740723,NaN,NaN,114.0461191534996,4.3784871101379395,142.2071533203125,NaN,...,2.295973969591319,NaN,NaN,NaN,NaN,NaN,FSB_130D,NaN,1383264.637,3875923.874
104850,2024-03-03 04:00:00.000,NaN,NaN,0.09242992848157883,NaN,NaN,114.05033618211746,4.378837585449219,142.18807983398438,NaN,...,2.2967315252818854,NaN,NaN,NaN,NaN,NaN,FSB_130D,NaN,1383264.637,3875923.874


,Date Time,Salinity (psu),Total Dissolved Solids (mg/L),station_id,Actual Conductivity (µS/cm),Specific Conductivity (µS/cm),Resistivity (Ω-cm),Density (g/cm³),Temperature (C),Pressure (psi),Level: Depth to Water (m),Depth (m),easting,northing
0,2023-04-01 05:00:00.000,0.09949345141649246,122.24631756544113,FSB_138D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1383153.232,3875722.045
1,2023-04-01 06:00:00.000,0.09974149614572525,122.54837900400162,FSB_138D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1383153.232,3875722.045
2,2023-04-01 07:00:00.000,0.10096397995948792,124.02405589818954,FSB_138D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1383153.232,3875722.045
3,2023-04-01 08:00:00.000,0.09829413890838623,120.80282717943192,FSB_138D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1383153.232,3875722.045
4,2023-04-01 09:00:00.000,0.09959174692630768,122.3672404885292,FSB_138D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1383153.232,3875722.045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237837,2024-03-03 01:00:00.000,0.01385036762803793,18.888847902417183,FSB_120D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1382808.060,3876214.780
237838,2024-03-03 02:00:00.000,0.013849947601556778,18.892895430326462,FSB_120D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1382808.060,3876214.780
237839,2024-03-03 03:00:00.000,0.01384719554334879,18.887918442487717,FSB_120D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1382808.060,3876214.780
237840,2024-03-03 04:00:00.000,0.013847293332219124,18.883513286709785,FSB_120D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1382808.060,3876214.780


In [ ]:
# testing some station id values
# looks good
desired_value = 'FEX_004'
result_df = obj.vulink_dfs[obj.vulink_dfs['station_id'] == desired_value]
result_df

,Date Time,Battery Level (%),Baro (psi),station_id,Temperature (C),Pulse (Hz),easting,northing
136706,2022-11-02 18:29:00.000,81.76890563964844,NaN,FEX_004,NaN,NaN,1383210.914,3875906.773
136707,2022-11-02 19:00:00.000,81.70653533935547,NaN,FEX_004,NaN,NaN,1383210.914,3875906.773
136708,2022-11-02 20:00:00.000,81.69367218017578,NaN,FEX_004,NaN,NaN,1383210.914,3875906.773
136709,2022-11-02 21:00:00.000,81.67759704589844,NaN,FEX_004,NaN,NaN,1383210.914,3875906.773
136710,2022-11-02 22:00:00.000,81.66281127929688,NaN,FEX_004,NaN,NaN,1383210.914,3875906.773
...,...,...,...,...,...,...,...,...
305765,2024-03-03 01:00:00.000,59.82823181152344,NaN,FEX_004,NaN,NaN,1383210.914,3875906.773
305766,2024-03-03 02:00:00.000,59.824283599853516,NaN,FEX_004,NaN,NaN,1383210.914,3875906.773
305767,2024-03-03 03:00:00.000,59.82060241699219,NaN,FEX_004,NaN,NaN,1383210.914,3875906.773
305768,2024-03-03 04:00:00.000,59.81718444824219,NaN,FEX_004,NaN,NaN,1383210.914,3875906.773


In [ ]:
obj.vulink_dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345576 entries, 0 to 345575
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Date Time          345576 non-null  object 
 1   Battery Level (%)  345576 non-null  object 
 2   Baro (psi)         173714 non-null  object 
 3   station_id         345576 non-null  object 
 4   Temperature (C)    348 non-null     object 
 5   Pulse (Hz)         7 non-null       object 
 6   easting            331922 non-null  float64
 7   northing           331922 non-null  float64
dtypes: float64(2), object(6)
memory usage: 21.1+ MB


## Save all dfs

    1. VuLink
    2. AT500
    3. AT200


In [ ]:
obj.vulink_dfs.to_csv("../data/csv_files/vulink.csv", index=False)
obj.at200_dfs.to_csv("../data/csv_files/at500.csv", index=False)
obj.at500_dfs.to_csv("../data/csv_files/at200.csv", index=False)